# Notebook create_osmnx_network_metropolis2020

This notebook runs the cities of metropolis>metro_gdf_2020, creates a Network (G,nodes,edges) for a bbox around the city and saves it to db

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Load data

In [2]:
# gdf_mun
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020'

query = f"SELECT * FROM {metro_schema}.{metro_table}"
metro_gdf = aup.gdf_from_query(query, geometry_col='geometry')
metro_gdf = metro_gdf.set_crs("EPSG:4326")

city_list = list(metro_gdf.city.unique())

# Show
print(metro_gdf.shape)
print(len(city_list))
print(city_list)
metro_gdf.head(1)

(367, 6)
71
['Aguascalientes', 'Ensenada', 'Mexicali', 'Tijuana', 'La Paz', 'Los Cabos', 'Campeche', 'Laguna', 'Monclova', 'Piedras Negras', 'Saltillo', 'Colima', 'Tapachula', 'Tuxtla', 'Chihuahua', 'Delicias', 'Juarez', 'CDMX', 'ZMVM', 'Durango', 'Celaya', 'Guanajuato', 'Leon', 'Irapuato', 'Acapulco', 'Chilpancingo', 'Pachuca', 'Tulancingo', 'Guadalajara', 'Vallarta', 'Piedad', 'Toluca', 'Morelia', 'Zamora', 'Uruapan', 'Cuautla', 'Cuernavaca', 'Tepic', 'Monterrey', 'Oaxaca', 'Puebla', 'San Martin', 'Tehuacan', 'Queretaro', 'Cancun', 'Chetumal', 'Playa', 'SLP', 'Culiacan', 'Los Mochis', 'Mazatlan', 'Guaymas', 'Ciudad Obregon', 'Hermosillo', 'Nogales', 'Villahermosa', 'Victoria', 'Matamoros', 'Nuevo Laredo', 'Reynosa', 'Tampico', 'Tlaxcala', 'Coatzacoalcos', 'Cordoba', 'Minatitlan', 'Orizaba', 'Poza Rica', 'Veracruz', 'Xalapa', 'Merida', 'Zacatecas']


,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,geometry,city
0,01001,01,001,Aguascalientes,"POLYGON ((-102.10641 22.06035, -102.10368 22.0...",Aguascalientes


## Run function create_osmnx_network for each city and save

In [ ]:
# prevent cities being analyzed several times in case of a crash
nodes_processed_city_list = []
edges_processed_city_list = []

try:
    query = f"SELECT city FROM osmnx.nodes_23_point"
    nodes_processed = aup.df_from_query(query)
    nodes_processed_city_list = list(nodes_processed.city.unique())
    
    query = f"SELECT city FROM osmnx.edges_23_line"
    edges_processed = aup.df_from_query(query)
    edges_processed_city_list = list(edges_processed.city.unique())

except:
    pass

# Done cities
i = len(edges_processed_city_list)

# Cities to go
j = len(city_list)

# Failed on:
    # Campeche: InterfaceError: (psycopg2.InterfaceError) connection already closed (Background on this error at: https://sqlalche.me/e/20/rvf5)
skip_city_list = []
#['ZMVM','CDMX']
#['Monterrey','Guadalajara','Campeche']

for city in city_list:
    if city not in skip_city_list:
        if (city not in nodes_processed_city_list) and (city not in edges_processed_city_list):
            print("--"*40)
            i = i + 1
            print(f"Starting city {i}/{j}: {city}")
        
            # Load area of interest
            city_gdf = metro_gdf.loc[metro_gdf.city == city]
            aoi = city_gdf.dissolve()
            
            # Create Network
            G,nodes,edges = aup.create_osmnx_network(aoi, how='from_bbox', network_type='all_private')

            nodes['city'] = city
            edges['city'] = city
        
            # Save Network
            print(f"Uploading nodes for {city}")
            aup.gdf_to_db_slow(nodes, 'nodes_23_point', 'osmnx', if_exists='append')
            print(f"Uploading edges for {city}")
            aup.gdf_to_db_slow(edges, 'edges_23_line', 'osmnx', if_exists='append')

        else:
            print("--"*40)
            print(f"{city} already processed. ({i}/{j})")
            
    else:
        print("--"*40)
        print(f"SKIPPED {city}")

--------------------------------------------------------------------------------
Aguascalientes already processed. (70/71)
--------------------------------------------------------------------------------
Ensenada already processed. (70/71)
--------------------------------------------------------------------------------
Mexicali already processed. (70/71)
--------------------------------------------------------------------------------
Tijuana already processed. (70/71)
--------------------------------------------------------------------------------
La Paz already processed. (70/71)
--------------------------------------------------------------------------------
Los Cabos already processed. (70/71)
--------------------------------------------------------------------------------
Campeche already processed. (70/71)
--------------------------------------------------------------------------------
Laguna already processed. (70/71)
--------------------------------------------------------------

In [ ]:
# Compare columns with db data
#city_list = ['Aguascalientes']

#for city in city_list:
    # Load area of interest
#    city_gdf = metro_gdf.loc[metro_gdf.city == city]
#    aoi = city_gdf.dissolve()

#    nodes_gdf = aup.gdf_from_polygon(aoi, 'osmnx', 'nodes', geom_col="geometry")
#    nodes_gdf = nodes_gdf.set_crs("EPSG:4326")
    
#    edges_gdf = aup.gdf_from_polygon(aoi, 'osmnx', 'edges', geom_col="geometry")
#    edges_gdf = edges_gdf.set_crs("EPSG:4326")

In [ ]:
# Check available data
#city_list = ['Aguascalientes']

#for city in city_list:
    # Load area of interest
#    city_gdf = metro_gdf.loc[metro_gdf.city == city]
#    aoi = city_gdf.dissolve()

#    nodes_gdf = aup.gdf_from_polygon(aoi, 'osmnx', 'nodes_23_point', geom_col="geometry")
#    nodes_gdf = nodes_gdf.set_crs("EPSG:4326")
    
#    edges_gdf = aup.gdf_from_polygon(aoi, 'osmnx', 'edges_23_line', geom_col="geometry")
#    edges_gdf = edges_gdf.set_crs("EPSG:4326")